# Sample Size Calculator

#### Steps:
 1) run the 3 cells under "Interactive calculator"  
 2) choose your conditions in third cell (if needed)  
 3) run the 2 cells under "Run everything below..." and wait for "Results" to populate


## Interactive calculator (user starts here)

In [47]:
import tubi_data_runtime as tdr
import math
import pandas as pd
import numpy as np
from datetime import date
# from statsmodels.stats.power import tt_ind_solve_power
from ipywidgets import interact, interactive, fixed, interact_manual, Button, HBox, VBox
import ipywidgets as widgets

from ssc_utils.filter_generator import filter_generator
from ssc_utils.raw_user_data import raw_user_data
from ssc_utils.metric_switcher import metric_switcher
from ssc_utils.metric_summary import metric_summary
from ssc_utils.cuped import cuped
import ssc_utils.calculator as c

# load choices
event_name_choices = filter_generator().event_name_choices()
filter_metrics_choices = filter_generator().filter_metrics_choices()

In [48]:
EFFECT_SIZE_RELATIVE = interactive(c.effect, effect=(0.0,1.0,0.01))
NUMBER_VARIATIONS = interactive(c.treatments, treatments=(0,8,1))
ALLOCATION = interactive(c.allocation, allocation=(0.0,1.0,0.01))
POWER = interactive(c.power, power=(0.0,1.0,0.01))
ALPHA = interactive(c.alpha, alpha=(0.0,1.0,0.01))

primary_metric = interactive(metric_switcher().choose_metric, metric = metric_switcher().possible_metrics())

attribute_filter = interactive(filter_generator().make_sql_condition_string, 
                                   field = filter_generator().filter_attributes_choices(), 
                                   condition = filter_generator().condition_choices(), 
                                   value = '', 
                                   filter_type = fixed('attribute'))

metric_filter = interactive(filter_generator().make_sql_condition_string, 
                                field = filter_metrics_choices, 
                                condition = filter_generator().condition_choices(), 
                                value = '', 
                                filter_type = fixed('metric'))


primary_event = widgets.SelectMultiple(
    options = event_name_choices,
    value = ('no event filter',), 
    description='Events'
#     disabled=False
)

primary_event_sub_cond = interactive(filter_generator().make_sql_condition_string, 
                              field = filter_generator().event_sub_cond_field_choices(), 
                              condition = filter_generator().condition_choices(), 
                              value = '', 
                              filter_type = fixed('event'))


pre_event = widgets.SelectMultiple(
    options = event_name_choices,
    value = ('no event filter',), 
    description='Events',
    disabled=False
)

pre_event_sub_cond = interactive(filter_generator().make_sql_condition_string, 
                              field = filter_generator().event_sub_cond_field_choices(), 
                              condition = filter_generator().condition_choices(), 
                              value = '', 
                              filter_type = fixed('event'))


time_interval = interactive(filter_generator().interval, interval = 'NULL')

In [49]:
parameters = widgets.VBox([EFFECT_SIZE_RELATIVE, NUMBER_VARIATIONS, ALLOCATION, POWER, ALPHA, primary_metric])

filter_accordion = widgets.Accordion([attribute_filter, metric_filter])
filter_accordion_titles = ['attribute', 'metric']
for i in range(0,len(filter_accordion_titles)):
    filter_accordion.set_title(i, filter_accordion_titles[i])

primary_event_set = VBox([primary_event, primary_event_sub_cond])
pre_event_set = VBox([pre_event, pre_event_sub_cond, time_interval])

event_filter_accordion = widgets.Accordion([primary_event_set, pre_event_set])
event_filter_accordion_titles = ['primary event','pre event']
for i in range(0,len(event_filter_accordion_titles)):
    event_filter_accordion.set_title(i, event_filter_accordion_titles[i])    

list_widgets  = [
    parameters,
    filter_accordion,
    event_filter_accordion   
]

tab = widgets.Tab(children = list_widgets)

titles = ['parameters','filters','event filters']
for i in range(0,len(titles)):
    tab.set_title(i, titles[i])
tab

## Run everything below after choosing your settings

In [78]:
filters_sql = filter_generator().generate_filter_cte(attribute_condition_interact = attribute_filter, 
                                                 metric_condition_interact = metric_filter, 
                                                 event1_condition_interact = pre_event, 
                                                 event1_sub_condition_interact = pre_event_sub_cond, 
                                                 event2_condition_interact = primary_event, 
                                                 event2_sub_condition_interact = primary_event_sub_cond, 
                                                 event_time_interval_interact = time_interval)
raw_user_sql = raw_user_data().generate_raw_user_data_cte(prev_cte_sql = filters_sql)
user_sql = metric_switcher().generate_user_data_cte(primary_metric.result) 
summary_sql = metric_summary().generate_metric_summary_cte() 
cuped_sql = cuped().generate_cuped_cte(event2_condition_interact = primary_event)

FINAL_SQL = filters_sql + raw_user_sql + user_sql + summary_sql + cuped_sql

# output SQL for debugging purposes; can copy and manually run this elsewhere
print(FINAL_SQL)


        WITH next_event AS ( 
          -- had to change this CTE name; for some reason "events" throws an error on Redshift... no idea why it works on Periscope
          -- TODO: figure out how to auto-sync this with our Periscope snippets 
          SELECT
            round((position/1000.0)/duration, 2) as content_completion_pct,
            a.device_id,
            a.user_id,
            a.platform,
            case
              when UPPER(a.platform) in (
                'IPHONE', 'IPAD', 'ANDROID', 'FIRETABLET', 'ANDROID-SAMSUNG', 'ANDROID_SAMSUNG', 'FOR_SAMSUNG', 'IOS_WEB', 'IOS') then 'MOBILE'
              when UPPER(a.platform) in (
                'WEB') then 'WEB'
              else
                'OTT'
            end
            as platform_type, -- TODO: figure out how to make this auto-sync with DBT: http://dw-docs.production-public.tubi.io/ux/#!/macro/macro.core_metrics.platform_type
            a.device_first_seen_ts as device_first_seen_ts,
            a.ts,
    

## Results

In [79]:
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

raw_df = tdr.query_redshift(FINAL_SQL).to_df()

final_df = c.calculate_sample_required(df = raw_df, 
                                       effect_size_relative = EFFECT_SIZE_RELATIVE, 
                                       number_variations = NUMBER_VARIATIONS, 
                                       allocation = ALLOCATION, 
                                       power = POWER, 
                                       alpha = ALPHA)

final_df.sort_values('platform').style\
    .hide_index()\
    .set_precision(3)\

/home/tubi/notebooks/shared/data_science/sample_size_calculator/ssc_utils/calculator.py:26: RuntimeWarning: invalid value encountered in true_divide
  std_effect_size = np.divide(mean_diff, sd_diff)


metric_name,platform,observations,avg_cuped_result,std_cuped_result,sample_required,weeks_required
signups,ALL,5837000.000,0.011,0.107,14151040,9.697
signups,AMAZON,795000.000,0.000,0.000,10,0.000
signups,ANDROID,844000.000,0.028,0.166,5368771,25.444
signups,COMCAST,316000.000,0.000,0.000,10,0.000
signups,COX,40000.000,0.000,0.000,10,0.001
signups,IPAD,133000.000,0.045,0.242,4516398,135.832
signups,IPHONE,303000.000,0.020,0.140,7794910,102.903
signups,MOBILE,1314000.000,0.027,0.168,5899301,17.958
signups,OTT,3793000.000,0.003,0.054,53985360,56.932
signups,PS4,123000.000,0.000,0.000,10,0.000
